## Extract, Transform, and Load Data using Python

### Objectives

* Read CSV, JSON, and XML file types.
* Extract the required data from the different file types.
* Transform data to the required format.
* Save the transformed data in a ready-to-load format   
  which can be loaded into an RDBMS.

### Importing Libraries and setting paths

The `ElementTree` function from the `xml.etree` library is used to parse the information from   
an `.xml` file format. The `.csv` and `.json` file formats can be read using the `pandas` library.

To call the correct function for data extraction, you need to access the file information using   
the `glob` library.

To log the information correctly, you need the date and time information at the point of logging   
and require the `datetime` package.

In [42]:
# glob, xml, and datetime are inbuilt features of Python
import glob
import pandas as pd
import xml.etree.ElementTree as ET
from datetime import datetime

In [43]:
# Create file paths
log_file = 'log_file.txt'  # to store all logs
target_file = 'transformed_data.csv'  # to store final output data

### Task 1: Extraction

In [44]:
# Function to extract data from CSV file format
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

In [45]:
'''Function to extract data from JSON file format. The 'lines=True' 
argument enables the file to be read on a line to line basis'''
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process, lines=True)
    return dataframe

To extract from an `XML` file, first parse the data from the file using the `ElementTree`   
function. Then extract the relevant information from this data and append it to a pandas   
dataframe. This data has headers for "name", "height", and "weight" for different persons.

In [46]:
# Function to extract data from XML file format
def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=["name", "height", "weight"])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for person in root:
        name = person.find("name").text
        height = float(person.find("height").text)
        weight = float(person.find("weight").text)
        dataframe = pd.concat([dataframe, pd.DataFrame([{"name":name, "height":height,"weight":weight}])], ignore_index=True)
    return dataframe

To call the relevant function to extract data, write a function which uses the `glob` library   
to identify the filetype.

In [47]:
def extract():
    extracted_data = pd.DataFrame(columns=['name', 'height', 'weight'])  # empty data frame to hold extracted data

    # process all csv files
    for csvfile in glob.glob("source/*.csv"):
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_csv(csvfile))], ignore_index=True)

    # process all json files
    for jsonfile in glob.glob("source/*.json"):
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_json(jsonfile))], ignore_index=True)

    # process all xml files
    for xmlfile in glob.glob("source/*.xml"):
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_xml(xmlfile))], ignore_index=True)

    return extracted_data

### Task 2: Tranformation

The height (in inches) and weight (in pounds) in the extracted data is required to be in meters   
and kilograms respectively.

In [48]:
# Function to perform the unit conversion for the two parameters
def transform(data):
    '''Convert inches to meters and round off to two decimals
    1 inch is 0.0254 meters '''
    data['height'] = round(data.height * 0.0254,2)

    '''Convert pounds to kilograms and round off to two decimals
    1 pound is 0.45359237 kilograms '''
    data['weight'] = round(data.weight * 0.45359237,2)

    return data

### Task 3: Loading and Logging

Load the transformed data to a `CSV` file that you can use to load to a database.

Write a function that accepts the transformed data as a dataframe and the `target_file` path.

In [49]:
def load_data(target_file, transformed_data):
    transformed_data.to_csv(target_file)

Finally, implement the logging operation to record the progress of the different operations, by   
recording a message, along with its timestamp, in the `log_file`.

In [50]:
# Write a function to record the message and its timestamp
def log_progress(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S'  # Year-Monthname-Day-Hour:Minute:Second
    now = datetime.now()  # get current timestamp
    timestamp = now.strftime(timestamp_format)  # convert timestamp to string format
    with open(log_file, "a") as f:
        f.write(timestamp + ', ' + message + '\n')

### Testing ETL operations and log progress

Test all functions and log the ETL process.

In [51]:
# Log the initialization of the ETL process
log_progress('ETL Job Started')

# Log the beginning of the Extraction process
log_progress('Extract phase Started')
extracted_data = extract()

# Log the completion of the Extraction process
log_progress('Extraction phase Ended')

# Log the beginning of the Transformation process
log_progress('Transform phase Started')
transformed_data = transform(extracted_data)  
print('Transformed Data')
print(transformed_data)

# Log the completion of the Transformation process
log_progress('Transform phase Ended')

# Log the beginning of the Loading process
log_progress('Load phase Started')
load_data(target_file, transformed_data)

# Log the completion of the Loading process
log_progress('Load phase Ended')

# Log the completion of the ETL process
log_progress('ETL Job Ended')

Transformed Data
     name  height  weight
0    alex    1.67   51.25
1    ajay    1.82   61.91
2   alice    1.76   69.41
3    ravi    1.73   64.56
4     joe    1.72   65.45
5    alex    1.67   51.25
6    ajay    1.82   61.91
7   alice    1.76   69.41
8    ravi    1.73   64.56
9     joe    1.72   65.45
10   alex    1.67   51.25
11   ajay    1.82   61.91
12  alice    1.76   69.41
13   ravi    1.73   64.56
14    joe    1.72   65.45
15   jack    1.74   55.93
16    tom    1.77   64.18
17  tracy    1.78   61.90
18   john    1.72   50.97
19   jack    1.74   55.93
20    tom    1.77   64.18
21  tracy    1.78   61.90
22   john    1.72   50.97
23   jack    1.74   55.93
24    tom    1.77   64.18
25  tracy    1.78   61.90
26   john    1.72   50.97
27  simon    1.72   50.97
28  jacob    1.70   54.73
29  cindy    1.69   57.81
30   ivan    1.72   51.77
31  simon    1.72   50.97
32  jacob    1.70   54.73
33  cindy    1.69   57.81
34   ivan    1.72   51.77
35  simon    1.72   50.97
36  jacob    1.70   5

C:\Users\mrlbh\AppData\Local\Temp\ipykernel_1580\1055280171.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_csv(csvfile))], ignore_index=True)
C:\Users\mrlbh\AppData\Local\Temp\ipykernel_1580\2894970490.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframe = pd.concat([dataframe, pd.DataFrame([{"name":name, "height":height,"weight":weight}])], ignore_index=True)
C:\Users\mrlbh\AppData\Local\Temp\ipykernel_1580\289